In [5]:
from datasets import load_dataset
import pandas as pd

In [16]:
# view the col context fully...

pd.set_option('display.max_colwidth', None)

In [6]:
ds = load_dataset("Amod/mental_health_counseling_conversations")

In [7]:
df = ds['train'].to_pandas()

In [8]:
df[:5]

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [17]:
unique_contexts = df['Context'].drop_duplicates()

In [18]:
len(unique_contexts)

995

In [29]:
# iloc is used to get specific index position of this dataframe. 

print(unique_contexts.iloc[5])

I have a severe back problem. I've had 3 major and several minor operations, but I'm still in constant pain. How can I deal with the depression from this chronic pain?
